In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_KOTA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C = individual[0]
    epsilon = 0.1

    if C <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,

In [ ]:
def main():
    # Setup DEAP framework
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    toolbox = base.Toolbox()
    
    # Register attributes and constants
    toolbox.register("attr_float", random.uniform)
    
    # Register individual and population
    toolbox.register("individual", tools.initCycle, creator.Individual, 
                        (lambda: random.uniform(*c_range),), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
    toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
    toolbox.register("evaluate", evaluate)

    c_ranges = [(0.0001, 0.01), (0.0001, 0.1), (0.01, 1), (0.01, 5), (1, 5), (1, 10), (10, 25), (10, 50)]
    crossover_rate = 0.7
    mutation_rate = 0.3
    population_size = 50
    num_generations = 200
    
    best_mape = float('inf')
    best_c_range = None
    
    results = []

    for c_range in c_ranges:        
        population = toolbox.population(n=population_size)

        for generation in range(num_generations):
            offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
            fits = toolbox.map(toolbox.evaluate, offspring)
            for fit, ind in zip(fits, offspring):
                ind.fitness.values = fit

            population = toolbox.select(offspring, k=len(population))

        best_individual = tools.selBest(population, k=1)[0]
        mape = best_individual.fitness.values[0]

        results.append((c_range, mape))
        print(f"MAPE terbaik untuk C range={c_range}: {mape}")

        if mape < best_mape:
            best_mape = mape
            best_c_range = c_range

    print(f"\nRentang C optimal: {best_c_range}")
    print(f"MAPE terbaik: {best_mape}")
    
    # Plot hasil
    fig, ax = plt.subplots()
    c_ranges_str = [f"{r[0]}-{r[1]}" for r in c_ranges]
    mapes = [r[1] for r in results]
    ax.plot(c_ranges_str, mapes, marker='o')
    ax.set_xlabel('C Range')
    ax.set_ylabel('MAPE')
    ax.set_title('MAPE vs C Range')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.show()

if __name__ == "__main__":
    main()

MAPE terbaik untuk C range=(0.0001, 0.01): 43.313402832555056
MAPE terbaik untuk C range=(0.0001, 0.1): 41.96605459669242
MAPE terbaik untuk C range=(0.01, 1): 36.80541133931505
MAPE terbaik untuk C range=(0.01, 5): 36.238148198635706
MAPE terbaik untuk C range=(1, 5): 36.02303803381516
